In [ ]:
import os
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import show_versions
plt.rcParams['figure.dpi'] = 100

In [ ]:
df = pd.read_csv("benchmarks/results/results.csv")

In [ ]:
commit = str(subprocess.check_output(['git', 'rev-parse', 'HEAD'])).replace("b'", "").replace("\\n'", "")[:10]

In [ ]:
cols = ["n_samples_train", "n_samples_test", "n_features", "n_neighbors"]
df[cols] = df[cols].astype(np.uint32)

In [ ]:
df_grouped = df.groupby(["n_samples_train", "n_samples_test", "n_features", "n_neighbors"])

In [ ]:
for i, (vals, df) in enumerate(df_grouped):
    # 16:9 ratio
    fig = plt.figure(figsize=(24, 13.5))
    ax = plt.gca()
    splot = sns.barplot(y="chunk_info", x="throughput", hue="implementation", data=df, ax=ax)
    _ = ax.set_xlabel("Thoughput (in GB/s)")
    _ = ax.set_ylabel("Chunk size (number of vectors), Number of X_train chunks")
    _ = ax.tick_params(labelrotation=45)
    for p in splot.patches:
        _ = splot.annotate(f"{p.get_width():.4e}", 
                       (p.get_width(), p.get_y() + p.get_height() / 2), 
                       ha = 'center', va = 'center', 
                       size=10,
                       xytext = (0, -12), 
                       textcoords = 'offset points')
    title = f"NearestNeighbors@{commit} - Euclidean Distance, dtype=np.float64, {df.trial.max() + 1} trials\n"
    title += ", ".join([f"{var}={os.environ.get(var)}" for var in ["OMP_NUM_THREADS", "OPENBLAS_NUM_THREADS", "MKL_NUM_THREADS"]]) + "\n"
    title += "n_samples_train=%s - n_samples_test=%s - n_features=%s - n_neighbors=%s" % vals
    _ = fig.suptitle(title, fontsize=16)
    plt.savefig(f"{i}.png")

### Machine specifications

In [ ]:
! cat /proc/version

In [ ]:
! lscpu

In [ ]:
! gcc -v

In [ ]:
! env

### Environment specifications

In [ ]:
show_versions()

In [ ]:
! conda list

---

In [ ]:
! date

----

### Cache analysis 

In [ ]:
out = str(subprocess.check_output(['getconf','-a'])).replace("b'", "").split("\\n")
cache_info = {}
for line in out:
    info = line.split(" ")
    if 'cache' in info[0].lower():
        cache_info[info[0]] = int(info[-1]) if info[-1] != '' else np.nan

In [ ]:
cache_info

In [ ]:
def what_fits(
    n: int,
    d: int,
    k: int,
    cache_info: dict,
    fdtype=np.float64,
    idtype=np.int64,
    verbose=False,
    parallel_on_xtrain=True,
):
    """ Quick analysis to see what fits in the caches.
    
    @param n: chunk_size
    @param d: number of features
    @param d: number of neighbours
    @param cache_info: machine caches' information
    @verbose cache_info: machine caches' information
    @parallel_on_xtrain: if True, analyse for the
        implementation parallelising on X_train, else
        analyse for the one on X_test
    """
    # dtype size
    sf = 8 if fdtype == np.float64 else 4
    si = 8 if idtype == np.int64 else 4
    
    # dist_middle_terms_chunks
    p_distance_matrix_size = n * n * sf
    
    # X_train_sq_norms
    x_train_sq_norms = n * sf
    
    # X_train[X_train_start:X_train_end, :] and
    # X_test[X_test_start:X_test_end, :]
    chunk_vects_size = 2 * n * d * sf
    
    # knn_indices and knn_red_distances
    chunk_heaps_size = n * k * (sf + si)
    
    # heaps_indices_chunks, needed for synchronisation
    extra_chunk_heaps_size = n * k * si if parallel_on_xtrain else 0

    total = (
        p_distance_matrix_size + 
        x_train_sq_norms + 
        chunk_vects_size + 
        chunk_heaps_size + 
        extra_chunk_heaps_size
    )
    
    print(f"Datastructure sizes for (n, d, k) = ({n}, {d}, {k}): {total} bytes")
    for level in ['LEVEL1_I', 'LEVEL1_D', 'LEVEL2_', 'LEVEL3_']:
        cache_size = cache_info[f'{level}CACHE_SIZE']
        if total < cache_size:
            print(f"    fits in {level} of size: {cache_size} bytes")
    
    if verbose:
        print(f"Datastructure sizes for (n, d, k) = ({n}, {d}, {k}):")
        print(f"Pairwise distance matrix :   {p_distance_matrix_size}")
        print(f"X_train squared norms    :   {x_train_sq_norms}")
        print(f"X and Y chunks vectors   :   {chunk_vects_size}")
        print(f"Heap on chunks           :   {chunk_heaps_size}")
        print(f"Extra heap for sync.     :   {extra_chunk_heaps_size}")
        print("-------------------------------------")
        print(f"Total                    :   {total}")
    print("")

In [ ]:
what_fits(128, 50, 100, cache_info)

In [ ]:
comb = df[['chunk_info', 'n_features', 'n_neighbors']]

In [ ]:
comb['chunk_size'] = comb['chunk_info'].apply(lambda s: int(s.split(',')[0].split('(')[1]))

In [ ]:
combinations = comb[['chunk_size', 'n_features', 'n_neighbors']].query('chunk_size > 0')

In [ ]:
for c in combinations.itertuples():
    what_fits(c.chunk_size, c.n_neighbors, c.n_neighbors, cache_info)